# Building a Spam Filter with Naive Bayes

## Introduction

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details [on this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

## Exploring the Dataset

We'll now start by reading in the dataset.

In [1]:
# Importing library
import pandas as pd

# Read in the dataset
spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [2]:
# Chacking information about dataset
spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
SMS      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


We see that our dataset contains twocolumns and 5,572 rows.

In [3]:
# Quick look in to dataset
spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [5]:
spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Abowe, we see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

## Training and Test Set

We're going to split our dataset intoo two categories:
- A **training set**, which we'll use to "train" the computer how to classify messages - these set will be contain 80% of data.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages - will be contain 20% of data.


In [6]:
# Randomize the dataset
dataset_randomize = spam.sample(frac=1, random_state=1)

In [7]:
# Calculate index for split
training_test_index = round(len(dataset_randomize) * 0.8)

# Training/Test split
training_set = dataset_randomize[:training_test_index].reset_index(drop=True)
test_set = dataset_randomize[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [8]:
training_set['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [9]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [10]:
test_set['Label'].value_counts()

ham     967
spam    147
Name: Label, dtype: int64

In [11]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The results look as we expected. We'll now move on to cleaning the dataset

## Data Cleaning

The next big step is to use the training set to teach the algorithm to classify new messages. But before we do taht we need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

### Letter Case and Punctuation

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [12]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [13]:
# Removing all the punctationform `SMS` column
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')

In [14]:
# Transform letter in to lower case
training_set['SMS'] = training_set['SMS'].str.lower()

In [15]:
#After cleaning
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [16]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [17]:
len(vocabulary)

7783

In our vocabulary we haave 7,783 words.

### The Final Training Set

Now we're going to use the vocabulary to make the data transformation we need. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

In [18]:
# Creating dictionary we need
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [19]:
# Transforming dictionary to DataFrame
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [20]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Applying Naive Bayes Classifier to Filter the Spam SMS

### Calculating Constants First

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

$$
P(Spam|w_1,w_2,...,w_n) ∝ P(Spam)⋅\prod_{i=0}^{n} P(w_i|Spam)
$$

$$
P(Ham|w_1,w_2,...,w_n) ∝ P(Ham)⋅\prod_{i=0}^{n} P(w_i|Ham)
$$

To calculate $P(w_i|Spam)$ and $P(w_i|Ham)$ we need to use below equations:

$$
P(w_i|Spam) = \frac {N_{w_i|Spam} + α}{N_{Spam} + α ⋅ N_{Vocabulary}}
$$

$$
P(w_i|Ham) = \frac {N_{w_i|Ham} + α}{N_{Ham} + α ⋅ N_{Vocabulary}}
$$

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:
- $P(Spam)$ and $P(Ham)$
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$

We'll also use Laplace smoothing and set $α=1$.

In [21]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

In [22]:
# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

In [23]:
# N_spam
n_word_per_spam = spam_messages['SMS'].apply(len)
n_spam = n_word_per_spam.sum()

In [24]:
# N_ham
n_word_per_ham = ham_messages['SMS'].apply(len)
n_ham = n_word_per_ham.sum()

In [25]:
# N_vocabulary
n_vocabulary = len(vocabulary)

In [26]:
# Laplace smoothing
alpha = 1

### Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham). Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

$$
P(w_i|Spam) = \frac {N_{w_i|Spam} + α}{N_{Spam} + α ⋅ N_{Vocabulary}}
$$

$$
P(w_i|Ham) = \frac {N_{w_i|Ham} + α}{N_{Ham} + α ⋅ N_{Vocabulary}}
$$

In [27]:
# Initialize dictionaries
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

In [28]:
# Calculate parameters
for word in vocabulary:
    # spam_messages already defined in a cell above
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    # ham_messages already defined in a cell above
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_given_ham

### Classifying A New Message

Now that we have all parameters calculated, we can start creating the spam filter.It can be  understood as a function that:
- Takes in as input a new message $(w_1, w_2, ..., w_n)$
- Calculates $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$
- Compares the values of $P(Spam|w_1, w_2, ..., w_n)$ and $P(Ham|w_1, w_2, ..., w_n)$, and:
    - If $P(Ham|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as ham.
    - If $P(Ham|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as spam.
    - If $P(Ham|w_1, w_2, ..., w_n) = P(Spam|w_1, w_2, ..., w_n)$, then the algorithm may request human help.

To write the code we need for calculating `p_spam_given_message` and `p_ham_given_message`, we need to use these two equations:

$$
P(Spam|w_1,w_2,...,w_n) ∝ P(Spam)⋅\prod_{i=0}^{n} P(w_i|Spam)
$$

$$
P(Ham|w_1,w_2,...,w_n) ∝ P(Ham)⋅\prod_{i=0}^{n} P(w_i|Ham)
$$

In [29]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower().split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Let's check our function.

In [30]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [31]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [32]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [33]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

$$
Accuracy = \frac{number of correctly classified messages}{total number of classified messages}
$$

In [35]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

## Conclusion

In this project, we built a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The naive Bayes classifier is a simple "probabilistic classifier" based on strong independence assumptions between the features, which is hardly true in the real world situations. The filter we created has an accuracy of 98.74% on the test set, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.